# Optional Example: Convert simulator backend to ONNX



In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

# Require:
#%pip install onnxruntime

import my_nb_path  # isort: split
import a2rl as wi
import numpy as np
import torch
from stable_baselines3 import A2C

from flight_sales.flight_sales_gym import flight_sales_gym

# Load artifacts
m = torch.load("model-dyn-pricing/model.pt")
tokenizer = wi.utils.pickle_load("model-dyn-pricing/tokenizer.pt")

# From data frame, get an input sequence
env = flight_sales_gym(f_reward="revenue_0_05")
env.reset()
env.step(0.5)
len_ar = len(tokenizer.df.actions) + len(tokenizer.df.rewards)
field_tokenizer = tokenizer.field_tokenizer
ctx = np.tile(
    field_tokenizer.transform(env.context(tail=2, fillna=True)).values.ravel()[:-len_ar],
    (1, 1),
)
ctx = torch.from_numpy(ctx)

# Export as ONNX format.
torch.onnx.export(
    m,
    ctx,
    "model-dyn-pricing/model.onnx",
    verbose=True,
    input_names=["input"],
    output_names=["output"],
    export_params=True,
    do_constant_folding=True,
)

# Optional next step: visualize with https://github.com/lutzroeder/netron.